PID Controller Notebook
* Ziegler-Nichols Method

Tutor:
* OpenAI's ChatGPT

In [12]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import odeint
from scipy.signal import find_peaks

"""
Abstract:
This script demonstrates the Ziegler-Nichols Closed-Loop Method for tuning a PID controller.
We simulate a first-order system (a simple example process) and apply the Ziegler-Nichols
method to tune the PID parameters. The system's response is plotted both before and after
tuning, and a table of the calculated PID gains is printed.
"""

# Function to model a first-order system (e.g., a thermal system)
def system_model(y, t, u, a, b):
    dydt = (-y + b * u) / a  # First-order system dynamics
    return dydt

# PID Controller class
class PIDController:
    def __init__(self, K_p, K_i, K_d):
        self.K_p = K_p
        self.K_i = K_i
        self.K_d = K_d
        self.prev_error = 0
        self.integral = 0

    # Calculate the PID output
    def update(self, error, dt):
        self.integral += error * dt
        derivative = (error - self.prev_error) / dt
        self.prev_error = error
        return self.K_p * error + self.K_i * self.integral + self.K_d * derivative

# System simulation with PID control
def simulate_system(K_p, K_i, K_d, t, y0, a, b):
    pid = PIDController(K_p, K_i, K_d)
    u = 0  # Control signal
    y = np.zeros_like(t)  # Output array
    y[0] = y0
    set_point = 1.0  # Desired setpoint (reference value)

    for i in range(1, len(t)):
        dt = t[i] - t[i-1]
        error = set_point - y[i-1]
        u = pid.update(error, dt)  # PID control signal
        dydt = system_model(y[i-1], t[i-1], u, a, b)  # Simplified model update
        y[i] = y[i-1] + dydt * dt  # Euler integration for system state update

    return y

# Find the ultimate gain and period using the closed-loop method
def find_ultimate_gain_period(t, y):
    peaks, _ = find_peaks(y, height=0.01, distance=10)
    peak_times = t[peaks]
    if len(peak_times) < 2:
        return None, None
    T_u = np.mean(np.diff(peak_times))  # Average time between peaks
    K_u = 1.0  # Manually set ultimate gain, adjust as needed
    return K_u, T_u

# Define simulation parameters
t = np.linspace(0, 20, 1000)  # Time array
y0 = 0  # Initial condition

# Function to prompt user for system parameters
def prompt_for_system_parameters():
    try:
        print("Please input system parameters for the first-order system:")
        print("Suggested value for time constant a: 2.0 (try between 0.5 and 5.0)")
        print("Suggested value for gain b: 1.0 (try between 0.5 and 2.0)")

        a = float(input("Enter the time constant 'a' for the system: "))
        b = float(input("Enter the gain 'b' for the system: "))

        return a, b

    except ValueError:
        print("Invalid input. Please enter numeric values for 'a' and 'b'.")
        return prompt_for_system_parameters()

# Function to prompt user for K_p_initial and suggest a range if necessary
def prompt_for_K_p(K_p_initial, a, b):
    try:
        while True:
            # Step 1: Find the Ultimate Gain (K_u) and Ultimate Period (T_u) by setting K_i and K_d to 0
            y_initial = simulate_system(K_p_initial, 0, 0, t, y0, a, b)

            # Find the ultimate gain and period
            K_u, T_u = find_ultimate_gain_period(t, y_initial)
            if K_u is not None and T_u is not None:
                return K_p_initial, y_initial, K_u, T_u

            # If oscillation does not occur, prompt user to input a new K_p value within a suggested range
            print(f"System did not oscillate with K_p_initial = {K_p_initial}.")
            print("Try increasing K_p_initial to a value in the range of 5 to 20 for oscillations.")
            print("Enter '-1' to exit.")

            new_K_p = float(input("Enter a new value for K_p_initial: "))

            if new_K_p == -1:
                print("Exiting the process gracefully.")
                return None, None, None, None  # Graceful exit condition
            else:
                K_p_initial = new_K_p

    except ValueError:
        print("Invalid input. Please enter a valid numeric value for K_p_initial.")
        return prompt_for_K_p(K_p_initial, a, b)

# Prompt for system parameters
a, b = prompt_for_system_parameters()

# Initial K_p value (can be adjusted manually if needed)
K_p_initial = 2.0
K_p_initial, y_initial, K_u, T_u = prompt_for_K_p(K_p_initial, a, b)

# Exit if the user decided to quit the loop
if K_p_initial is None:
    print("Tuning process terminated by the user.")
else:
    # Step 2: Apply Ziegler-Nichols Tuning Rules for PID
    K_p_zn = 0.6 * K_u
    K_i_zn = 2 * K_p_zn / T_u
    K_d_zn = K_p_zn * T_u / 8

    # Step 3: Simulate the system with the tuned PID values
    y_tuned = simulate_system(K_p_zn, K_i_zn, K_d_zn, t, y0, a, b)

    # Output the table of PID values
    pid_table = pd.DataFrame({
        'Parameter': ['K_p', 'K_i', 'K_d'],
        'Value': [K_p_zn, K_i_zn, K_d_zn]
    })
    print("Tuned PID Parameters (Ziegler-Nichols Method):")
    print(pid_table.to_string(index=False))

    # Plot the system response before and after tuning
    plt.figure(figsize=(10, 6))
    plt.plot(t, y_initial, label=f'Before Tuning (K_p = {K_p_initial:.2f}, K_i = 0, K_d = 0)', linestyle='--')
    plt.plot(t, y_tuned, label=f'After Tuning (K_p = {K_p_zn:.2f}, K_i = {K_i_zn:.2f}, K_d = {K_d_zn:.2f})', linewidth=2)
    plt.axhline(1.0, color='gray', linestyle=':', label='Set Point')
    plt.title('System Response Before and After PID Tuning (Ziegler-Nichols Method)')
    plt.xlabel('Time')
    plt.ylabel('System Output')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Output the PID tuning results in a table format
    print("\nTable of System Responses:")
    response_table = pd.DataFrame({
        'Time (s)': t,
        'Before Tuning Output': y_initial,
        'After Tuning Output': y_tuned
    })
    print(response_table.head())  # Display first few rows of output table


Please input system parameters for the first-order system:
Suggested value for time constant a: 2.0 (try between 0.5 and 5.0)
Suggested value for gain b: 1.0 (try between 0.5 and 2.0)
Enter the time constant 'a' for the system: 3
Enter the gain 'b' for the system: 1.2
System did not oscillate with K_p_initial = 2.0.
Try increasing K_p_initial to a value in the range of 5 to 20 for oscillations.
Enter '-1' to exit.
Enter a new value for K_p_initial: 6
System did not oscillate with K_p_initial = 6.0.
Try increasing K_p_initial to a value in the range of 5 to 20 for oscillations.
Enter '-1' to exit.
Enter a new value for K_p_initial: -1
Exiting the process gracefully.
Tuning process terminated by the user.
